# Einführung in Predictive Business Process Monitoring

## Definition + Ziel
Predictive Business Process Monitoring (PBPM) beinhaltet die Anwendung von Data-Mining- und Predictive-Analytics Techniken auf **Eventlogs**.

Eventlogs entstehen durch das automatische Protokollieren von Aktivitäten, Zeitstempeln und Attributen in Informationssystemen und sind die Grundlage für die Analyse und Optimierung von Geschäftsprozessen.

Ziel von PBPM ist die Vorhersage zukünftiger Eigenschaften laufender Prozesse durch Analyse und Modellierung von Mustern und Sequenzen in historischen Daten.


## Strukture der Daten

In einem typischen Eventlog findet man zumindest eine **case_id**, eine **activity** und einen **timestamp**. Zusätzlich sind öfter auch weitere **context** attribute zu finden.

Prozessdaten können in einer Tabelle dargestellt werden, unterscheiden sich jedoch in der Representation von tabellarischen Daten.

Erkennbar ist, dass im Gegensatz zu tabellarischen Daten, die in Zeilen (pro Zeile ein Sample) und Spalten (Features) organisiert sind, Eventlogs in Form von **Traces** oder auch Instanzen vorliegen.

## Definition eines Samples und Features in einem Eventlog

Hier gehören also mehrere Zeilen zu einem Sample. Ein Trace ist eine Sequenz von Ereignissen,
welches die Aktivitäten eines Prozesses in der Reihenfolge ihres Auftretens darstellt.

Das bedeutet, ein Sample besteht nicht aus einer Zeile, sondern aus einer Sequenz von Aktivitäten, die in einer bestimmten Reihenfolge stattgefunden haben.

## Abtrennung von Prozess Instanzen durch eine Instanz ID / Case ID

Markiert wird ein Trace oft durch eine **case_id**.
Unsere Aufgabe ist es also, dem Modell einen (Teil) Trace zu geben und es soll uns sagen, was als nächstes passiert.

Das Label ist also die nächste Aktivität, die in dem Trace passiert oder auch die Zeit, die bis zum Abschluss des Prozesses verbleibt.

Es wird deutlich, dass einige Vorverarbeitung notwendig ist, um die Daten in ein Format zu bringen, das für Machine Learning Modelle geeignet ist.

<img src="event log example.png">

## Wiederholung Process Mining

Die Übersetzung in ein mögliches Prozessmodell in Form eines Graphs (hier links) sieht in dem Auszug relativ simpel und per Hand machbar aus.

Sobald die Anzahl der Ereignisse und die Anzahl der Traces (Instanzen des Prozess) steigt, wird es jedoch schwierig, die Beziehungen zwischen den Ereignissen zu erkennen und zu modellieren. Ab einem gewissen Punkt brauchen wir Abstraktionen und Vereinfachung. Oft werden diese in verschiedenen Stufen durch verschiedene Algorithmen erreicht.

## Existierende Klassifikations und Regressions Probleme in PBPM

PBPM zielt darauf ab, Folgendes vorherzusagen:

* nächste Aktivitäten an einem bestimmten Punkt in einem Prozess
* (z.B. welche Aktivität wird als nächstes ausgeführt).
* den Ausgang von Fällen (positiv / negativ, z.B. Annahme eines Kredits oder Ablehnung, ...).
* die verbleibende Zeit bis zum Abschluss eines Falles (z.B. wie lange wird ein Kunde noch warten müssen).
* andere relevante Fallattribute (z.B. die Wahrscheinlichkeit, dass ein Fall in einer bestimmten Abteilung bearbeitet wird).

Diese Vorhersagen können die betriebliche Effizienz erheblich steigern, die Kundenzufriedenheit verbessern und die proaktive Entscheidungsfindung in verschiedenen Geschäftsprozessen erleichtern. Bis dahin ist es aber oft ein weiter Weg und Firmen müssen sich mit den Herausforderungen der Datenqualität, der Modellierung und der Integration in bestehende Systeme auseinandersetzen.

Oft werden Tools wie von Celonis oder PM4PY angewandt, da diese speziell für die Prozessanalyse und -optimierung entwickelt wurden.
Diese Tools inkludieren oft aber nur Analyse Werkzeuge.

Für die Vorhersage von Prozessen werden oft Machine Learning Modelle verwendet, die in Python oder R entwickelt werden und Programmierkenntnisse voraussetzen.

Historisch gibt es einige Low-Code Platformen oder Pakete, die uns aber nur bedingt beim Verständnis und der Anwendung von Machine Learning Modellen helfen.

Im Folgenden wollen wir verstehen, wie wir mit einfachen Machine Learning Modellen in Python Eigenschaften von laufenden Prozesen vorhersagen können.

## Fallbeispiel + Daten Exploration
### HelpDesk Dataset

* Das Eventlog "Helpdesk" stammt aus dem Ticketing-Management-System des Helpdesks (Kundensupports) eines italienischen Softwareunternehmens.
* Dieses Prozessprotokoll ist die Grundlage für die Untersuchung des Lebenszyklus von Helpdesk-Tickets, von der Erstellung bis zur Lösung.
* Der Prozess, den wir aus diesem Eventlog ableiten können, besteht aus 14 verschiedenen Aktivitäten. Der Datensatz enthält 4580 Instanzen und 13.710 Ereignisse, mit einer Länge von 1 bis 14 Aktivitäten.
* Die Analyse dieses Protokolls hilft dabei, den Ticketprozess zu verstehen und zu verbessern, um die Effizienz und Kundenzufriedenheit im Helpdesk-Betrieb zu steigern.
* Die Daten zu diesem öffentlichen Datensatz finden Sie [hier](https://data.4tu.nl/articles/dataset/Dataset_belonging_to_the_help_desk_log_of_an_Italian_Company/12675977
* Der Zeitstempel hilft zunächst bei der Analyse der Dauer der einzelnen Aktivitäten und des gesamten Prozesses. Ein Machine Learning modelle kann dann auf Basis dieser Daten trainiert werden, um zukünftige Prozesse zu prognostizieren.

### Traces und ihre Visualisierung
- Auswahl einiger Traces und manuelle Überführung in einen Graphen
- Diskussion über die Bedeutung der Reihenfolge von Events in einem Prozess

## We start off our Coding with some simple imports of packages that we will need

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process.kernels import RBF
import time

# import datev event log data
# which consists of a string with "case;event;time;DOMAIN;SUBSYSTEM;MICROSUBSYSTEM;EVENTTAX_5;EVENTTAX_6;EVENTTAX_7"
# case event and time are relevant and
from sklearn.neighbors import KNeighborsClassifier

eventlog = pd.read_csv('data_converted.csv', delimiter=';')
#eventlog = pd.read_csv('helpdesk_converted.csv', delimiter=';')
eventlog = eventlog.drop(columns=['time'])

#modify data (datanew = data + 1)for calculation of gaussian process later, because event 0 is going to be a filler/blank
#LATER this would be reversed if a system in practical use cases would be needed
column_names = eventlog.columns ##take columns of a typical event log except the case column and increment
for i in range(1, len(column_names)):
    eventlog[column_names[i]] += 1

maxeventseq = (eventlog.groupby(['case']).count()['event']).max()

#now Build sample arrays for each case: for each k events do this k - 1 times
case_groups = eventlog.groupby('case')

samples_to_map = [] #contains a sequence of all samples in the event log, each sample is a 2 dimensional array of rows of event, domains and is filled with the number 0 for each row missing
labels = [] #1 dimensional array

for case, group in case_groups:
    for n in range(len(group) - 1): #create n - 1 samples in this group
        sample = [] #[maxeventseq][len(group.columns)] at the end
        for i in range(n + 1): #use i to current n amount of events
            sample_row = []
            for r in range(1, len(group.columns)):
                sample_row.append(group.iloc[i, r])
            sample.append(sample_row)
        for i_fillUp in range(n + 1, maxeventseq): #fill n to < maxeventseq with blank/0 event  ###minus 1 in range because of the last max event seq is going to be the lable
            sample_row = []
            for r in range(1, len(group.columns)):
                sample_row.append(0)
            sample.append(sample_row)

        samples_to_map.append(sample)
        labels.append(group.iloc[n + 1, 1])

#map the samples arra from 3d to 2d by concating every sample
samples = [] #array of mapped samples
for sample in samples_to_map:
    mapped_sample = []
    for sample_row in sample:
        for sample_feature in sample_row:
            mapped_sample.append(sample_feature)
    samples.append(mapped_sample)

samples_np = np.array(samples, dtype=int)
labels_np = np.array(labels, dtype=int)

trainsize = 0.70
testsize = 0.30
#Classification happens here
Sample_train, Sample_test, feature_train, feature_test = train_test_split(samples_np,
                                                                          labels_np,
                                                                          train_size=trainsize,
                                                                          test_size=testsize,

                                                                          )

start = time.time()

#kernel = 1.0 * RBF(length_scale=1.0)## [1.0] ##kernel=kernel #ConstantKernel(1.0) * RBF(length_scale=1.0)
#kernel = 0.75 * RBF([0.75])
gpc = RandomForestClassifier().fit(Sample_train, feature_train) #RandomForestClassifier #DecisionTreeClassifier(max_depth=5) ##KNeighborsClassifier(3)
#gpc = GaussianProcessClassifier().fit(Sample_train, feature_train) #, n_jobs=4
print('gpc done')
stop = time.time()
duration = int(stop - start)
print(duration)

#print options
np.set_printoptions(threshold=np.inf)
# calculate
accuracy =gpc.score(Sample_test, feature_test) # gpc

print(accuracy)

# Confusion Matrix
gpc_predictions = gpc.predict(Sample_test) #gpc
cm = confusion_matrix(feature_test, gpc_predictions)
#print(cm)

result_cf = pd.DataFrame(cm)
result_ac_time = pd.DataFrame([[accuracy, duration, trainsize, testsize
                                #, kernel.k1
                                ]], columns=['accuracy', 'runtime_in_s', 'training', 'testing'
                                        # , 'kernel'
                                            ]
                              )
result_cf.to_csv('result_conf_m.csv', index=False, sep=';')
result_ac_time.to_csv('result_accur_time.csv', index=False, sep=";")

In [ ]:
# Sometimes we want to set options for printing the output in our console to show the entire dataframe.
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 20)

PM4Py is a process mining library in Python that offers a wide range of tools and algorithms for the analysis, discovery, and enhancement of processes. Process mining is a field that focuses on the analysis of event logs to discover, monitor, and improve real processes by extracting knowledge from event logs readily available in today's systems. PM4Py implements various algorithms and techniques for process discovery, conformance checking, and enhancement, catering to the needs of process mining tasks.

The differences between the Directly Follows Graph (DFG) miner, the Alpha Miner, the Inductive Miner, and the Heuristics Miner are primarily in their approach to discovering process models from event logs, each with its strengths and limitations:

Directly Follows Graph Miner: This miner constructs a graph where nodes represent activities, and edges represent the direct succession of one activity by another in the event log. It's a simple and intuitive model, highlighting the direct relationships between activities. However, it might not capture more complex dependencies and can result in cluttered graphs for processes with many activities.
Alpha Miner: One of the earliest and most straightforward process discovery algorithms, the Alpha Miner, can construct a Petri net (a mathematical modeling language) from an event log. It identifies causal dependencies between activities based on their order of appearance. While effective for simple, well-structured processes, it struggles with more complex scenarios, such as loops of varying lengths and parallel executions.
Inductive Miner: This miner is designed to handle a wider variety of process behaviors, including complex structures like loops and parallel activities. It works by recursively splitting the log into simpler parts until it can apply base cases to construct a process model. The Inductive Miner is particularly known for its balance between accuracy and simplicity, providing models that are both readable and close to the event log data. It also has a variant that can guarantee sound process models, which are free from deadlocks and other anomalies.
Heuristics Miner: This algorithm focuses on discovering process models based on heuristics derived from the event log. It aims to balance between overfitting and underfitting the log data by considering frequencies of activity occurrences and their relations. The Heuristics Miner is particularly useful for dealing with noisy data or logs with incomplete information. It constructs a process model that reflects the most common behavior observed in the event log, while potentially ignoring rare or exceptional paths.

Ab einem gewissen Punkt brauchen wir Abstraktionen und Vereinfachung. Oft werden diese in verschiedenen Stufen durch verschiedene Algorithmen erreicht.
Beispiel dafür sind von PM4PY:
* Directly Follows Graph Miner (DFG). Hier werden die direkten Beziehungen zwischen den Ereignissen in einem Graphen dargestellt. Die Knoten repräsentieren Aktivitäten und die Kanten repräsentieren die direkte Abfolge von Aktivitäten. Oft können Zyklische Abhängigkeiten und parallele Ausführungen nicht dargestellt werden.
* Alpha Miner. Hier wird ein Petri Netz aus einem Ereignisprotokoll erstellt. Es identifiziert kausale Abhängigkeiten zwischen Aktivitäten basierend auf ihrer Reihenfolge des Auftretens. Dieser Algorithmus ist effektiv für einfache, gut strukturierte Prozesse, hat aber Schwierigkeiten mit komplexeren Szenarien, wie Schleifen unterschiedlicher Längen und parallelen Ausführungen.
* Inductive Miner. Dieser Miner ist darauf ausgelegt, eine breitere Vielfalt von Prozessverhalten zu handhaben, einschließlich komplexer Strukturen wie Schleifen und parallelen Aktivitäten. Er arbeitet, indem er das Protokoll rekursiv in einfachere Teile aufteilt, bis er Basisszenarien anwenden kann, um ein Prozessmodell zu erstellen. Der Inductive Miner ist bekannt für sein Gleichgewicht zwischen Genauigkeit und Einfachheit und liefert Modelle, die sowohl lesbar als auch nahe an den Ereignisprotokolldaten sind.
* Heuristics Miner. Dieser Algorithmus konzentriert sich auf die Entdeckung von Prozessmodellen auf der Grundlage von Heuristiken, die aus dem Ereignisprotokoll abgeleitet wurden. Er zielt darauf ab, zwischen Überanpassung und Unteranpassung der Protokolldaten zu balancieren, indem er die Häufigkeiten des Auftretens von Aktivitäten und deren Beziehungen berücksichtigt. Der Heuristics Miner ist besonders nützlich für den Umgang mit rauschhaften Daten oder Protokollen mit unvollständigen Informationen. Er erstellt ein Prozessmodell, das das häufigste Verhalten im Ereignisprotokoll widerspiegelt, wobei möglicherweise seltene oder außergewöhnliche Pfade ignoriert werden.

## Projekt: Case 2 - BPI Challenge 2012 (BPI12 W Complete) Event Log

The BPI Challenge 2012 event log details an application process for personal loans or overdrafts within a global financing organization, encompassing three subprocesses related to application states, work item states, and their interactions. To focus on manually executed activities, the dataset is pre-processed to exclude automatic events, resulting in 9,658 work item traces and 72,413 "complete" type events across 6 activities. This preprocessing, consistent with prior studies, allows for meaningful comparisons and insights into the manual aspects of the loan application process, with trace lengths varying between 1 and 74.